Install Necessary Libraries

In [5]:
# Install Hugging Face Transformers and Datasets libraries
!pip install transformers datasets

# Install Streamlit and Gradio for optional web apps
!pip install streamlit gradio


Load a Pre-Trained Model

In [6]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load the model and tokenizer
model_name = "bigscience/bloom-560m"  # A smaller version of Bloom
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

print("Model and tokenizer loaded successfully!")


Model and tokenizer loaded successfully!


Test the Model

In [7]:
# Define a function to generate answers
def generate_answer(question, max_length=100):
    inputs = tokenizer(question, return_tensors="pt")
    outputs = model.generate(inputs["input_ids"], max_length=max_length, num_return_sequences=1)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Test the model
question = "What is artificial intelligence?"
answer = generate_answer(question)
print("Question:", question)
print("Answer:", answer)


Question: What is artificial intelligence?
Answer: What is artificial intelligence? Artificial intelligence is a new technology that is used to solve problems in the field of computer science. Artificial intelligence is a new technology that is used to solve problems in the field of computer science. Artificial intelligence is a new technology that is used to solve problems in the field of computer science. Artificial intelligence is a new technology that is used to solve problems in the field of computer science. Artificial intelligence is a new technology that is used to solve problems in the field of computer science.


Add Bangla Support

In [8]:
# Test with a Bangla question
bangla_question = "কৃত্রিম বুদ্ধিমত্তা কী?"
bangla_answer = generate_answer(bangla_question)
print("প্রশ্ন:", bangla_question)
print("উত্তর:", bangla_answer)


প্রশ্ন: কৃত্রিম বুদ্ধিমত্তা কী?
উত্তর: কৃত্রিম বুদ্ধিমত্তা কী?


##  Fine-Tuning the Model

Prepare the Dataset

In [9]:
{"question": "What is AI?", "answer": "AI stands for Artificial Intelligence."}
{"question": "কৃত্রিম বুদ্ধিমত্তা কী?", "answer": "কৃত্রিম বুদ্ধিমত্তা মানে হল কৃত্রিমভাবে তৈরি বুদ্ধি।"}


{'question': 'কৃত্রিম বুদ্ধিমত্তা কী?',
 'answer': 'কৃত্রিম বুদ্ধিমত্তা মানে হল কৃত্রিমভাবে তৈরি বুদ্ধি।'}

Save it as qa_dataset.jsonl in Colab:

In [10]:
import json

data = [
    {"question": "What is AI?", "answer": "AI stands for Artificial Intelligence."},
    {"question": "কৃত্রিম বুদ্ধিমত্তা কী?", "answer": "কৃত্রিম বুদ্ধিমত্তা মানে হল কৃত্রিমভাবে তৈরি বুদ্ধি।"}
]

with open("qa_dataset.jsonl", "w") as f:
    for entry in data:
        f.write(json.dumps(entry) + "\n")

print("Dataset saved as qa_dataset.jsonl")


Dataset saved as qa_dataset.jsonl


Fine-Tune the Model

In [11]:
!pip install accelerate

Fine-tuning code:

In [13]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("json", data_files="qa_dataset.jsonl")

# Prepare data for fine-tuning
def preprocess_data(example):
    return {"input_ids": tokenizer(example["question"], truncation=True, padding="max_length", max_length=128)["input_ids"],
            "labels": tokenizer(example["answer"], truncation=True, padding="max_length", max_length=128)["input_ids"]}

tokenizer.pad_token = tokenizer.eos_token  # Set padding token
tokenized_dataset = dataset.map(preprocess_data, remove_columns=["question", "answer"])

# Fine-tuning setup
training_args = TrainingArguments(
    output_dir="./fine_tuned_model",
    overwrite_output_dir=True,
    per_device_train_batch_size=2,
    num_train_epochs=1,
    save_steps=10,
    save_total_limit=1,
    logging_dir='./logs',
    logging_steps=10,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
)

# Fine-tune the model
trainer.train()


Map:   0%|          | 0/2 [00:00<?, ? examples/s]

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: ERROR API key must be 40 characters long, yours was 45


...

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss


TrainOutput(global_step=1, training_loss=5.131471633911133, metrics={'train_runtime': 307.3951, 'train_samples_per_second': 0.007, 'train_steps_per_second': 0.003, 'total_flos': 464353492992.0, 'train_loss': 5.131471633911133, 'epoch': 1.0})

Save the Fine-Tuned Model

In [14]:
trainer.save_model("./fine_tuned_model")
print("Model saved successfully!")


Model saved successfully!


Test the Fine-Tuned Model

In [15]:
# Load the fine-tuned model
fine_tuned_model = AutoModelForCausalLM.from_pretrained("./fine_tuned_model")

# Test with a question
question = "কৃত্রিম বুদ্ধিমত্তা কী?"
inputs = tokenizer(question, return_tensors="pt")
outputs = fine_tuned_model.generate(inputs["input_ids"], max_length=100, num_return_sequences=1)
print("প্রশ্ন:", question)
print("উত্তর:", tokenizer.decode(outputs[0], skip_special_tokens=True))


প্রশ্ন: কৃত্রিম বুদ্ধিমত্তা কী?
উত্তর: কৃত্রিম বুদ্ধিমত্তা কী?
